In [1]:
import sys
sys.path.append('../..')

In [2]:
from src.recommenders.baseline_recommender import BaselineRecommender
from src import evaluation as eval

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm

from sklearn.model_selection import train_test_split

# Load the data

In [4]:
books_df = pd.read_csv('../../data/Books.csv', delimiter=';')
ratings_df = pd.read_csv('../../data/Ratings.csv', delimiter=';')
users_df = pd.read_csv('../../data/Users.csv', delimiter=';')

/var/folders/s0/4m_t_82936ggv17n0g0t7_dr0000gn/T/ipykernel_41158/2786065433.py:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  users_df = pd.read_csv('../../data/Users.csv', delimiter=';')


# Train-test split

In [5]:
train_ratings, test_ratings = train_test_split(ratings_df, test_size=0.3, random_state=42)

# Model Initialization

In [6]:
model = BaselineRecommender()
model.fit(items=books_df, users=users_df, ratings=train_ratings)

# Model prediction

In [7]:
test_users = test_ratings['User-ID'].unique()

In [8]:
best_items = model.predict(test_users)

In [9]:
best_items[0]

['0439139597',
 '0439064864',
 '0439136350',
 '0590353403',
 '043935806X',
 '0812550706',
 '0553375407',
 '0439136369',
 '0440219078',
 '0439139600']

In [10]:
len(test_users)

53334

# Model evaluation

In [11]:
# relevant items are those with rating >= 5
rating_mask = test_ratings['Rating'] >= 5

# init arrays to store the results
avg_p = []
prec_k = []
rec_k = []

# iterate over test users and predicted items
for user, items in tqdm(zip(test_users, best_items)):
    # get relevant items for the user
    user_mask = test_ratings['User-ID'] == user
    relevant_items = test_ratings.loc[(user_mask & rating_mask)].sort_values(by='Rating', ascending=False)
    relevant_items = relevant_items['ISBN'].to_numpy()
    if len(relevant_items) == 0:
        continue
    # calculate metrics
    ap = eval.average_precision(items, relevant_items, m=10)
    pk = eval.precision_at_k(items, relevant_items, k=10)
    rk = eval.recall_at_k(items, relevant_items, k=10)
    avg_p.append(ap)
    prec_k.append(pk)
    rec_k.append(rk)

# calculate mean metrics over test users
mean_ap = np.mean(avg_p)
mean_prec_k = np.mean(prec_k)
mean_rec_k = np.mean(rec_k)

print(f'Mean Average Precision: {mean_ap}')
print(f'Mean Recall at 10: {mean_rec_k}')
print(f'Mean Precision at 10: {mean_prec_k}')

53334it [00:16, 3160.94it/s]

Mean Average Precision: 0.0013982744421767394
Mean Recall at 10: 0.0028529763043792992
Mean Precision at 10: 0.0010444365251240442
